In [30]:
import re

In [18]:
import requests

url = "https://www.digikala.com/ajax/product/comments/507136/"

querystring = {"page":"1","mode":"buyers"}
# querystring = {"page":"1","mode":"most_liked"}

headers = {
    'User-Agent': "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0",
    'Accept': "*/*",
    'Accept-Language': "en-US,en;q=0.5",
    'Referer': "https://www.digikala.com/product/dkp-507136/%D9%84%D8%A7%D9%85%D9%BE-%D8%A7%D9%84-%D8%A7%DB%8C-%D8%AF%DB%8C-25-%D9%88%D8%A7%D8%AA-%D8%A8%D8%B1%D9%88%DA%A9%D8%B3-%D9%85%D8%AF%D9%84-1739-a95-%D9%BE%D8%A7%DB%8C%D9%87-e27?variant_id=956818",
    'Content-Type': "application/x-www-form-urlencoded; charset=UTF-8",
    'X-Requested-With': "XMLHttpRequest",
    'Connection': "keep-alive",
    'Cache-Control': "no-cache",
    'Postman-Token': "3dbd5dca-72df-4f38-862c-8f0c1edad5ee,2ea246f4-01fb-4652-ba3c-768f3641539f",
    'Host': "www.digikala.com",
    'Accept-Encoding': "gzip, deflate",
    'cache-control': "no-cache"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

In [19]:
response.text

'    <h2 class="c-comments__headline">امتیاز کاربران به:\n        <span>\n    <span>\n        لامپ ال ای دی 25 وات بروکس مدل 1739-A95 پایه E27</span><span> |</span>\n    <span>۳</span><span> / ۵</span>\n        <span>(۴۰۰ نفر)\n    </span>\n    </span>\n    </h2>\n\n<div class="c-comments__summary">\n            <div class="c-comments__summary-box">\n            <ul class="c-comments__item-rating">\n                                    <li>\n                        <div class="cell">طراحی:</div>\n                        <div class="cell">\n                                                                                                                                                <div class="c-rating c-rating--general js-rating"\n                                 data-rate-digit="معمولی">\n                                <div class="c-rating__rate js-rating-value" data-rate-value="68.6%"></div>\n                            </div>\n                        </div>\n                    </li

In [20]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

print(soup.prettify())

<h2 class="c-comments__headline">
 امتیاز کاربران به:
 <span>
  <span>
   لامپ ال ای دی 25 وات بروکس مدل 1739-A95 پایه E27
  </span>
  <span>
   |
  </span>
  <span>
   ۳
  </span>
  <span>
   / ۵
  </span>
  <span>
   (۴۰۰ نفر)
  </span>
 </span>
</h2>
<div class="c-comments__summary">
 <div class="c-comments__summary-box">
  <ul class="c-comments__item-rating">
   <li>
    <div class="cell">
     طراحی:
    </div>
    <div class="cell">
     <div class="c-rating c-rating--general js-rating" data-rate-digit="معمولی">
      <div class="c-rating__rate js-rating-value" data-rate-value="68.6%">
      </div>
     </div>
    </div>
   </li>
   <li>
    <div class="cell">
     کیفیت ساخت:
    </div>
    <div class="cell">
     <div class="c-rating c-rating--general js-rating" data-rate-digit="معمولی">
      <div class="c-rating__rate js-rating-value" data-rate-value="68.4%">
      </div>
     </div>
    </div>
   </li>
   <li>
    <div class="cell">
     راندمان مصرف انرژی:
    </div>
    <d

In [21]:
soup.find(class_="c-comments__list").findAll(class_="aside")

[<div class="aside">
 <ul class="c-comments__user-shopping">
 <li>
 <div class="cell cell-name">
                                 حمید اکبری
                             </div>
 <div class="c-comments__buyer-badge">خریدار</div>
 </li>
 <li>
 <div class="cell">
                                 در تاریخ ۱۰ مرداد ۱۳۹۸
                             </div>
 </li>
 </ul>
 <div class="c-message-light c-message-light--opinion-negative">خرید این محصول را توصیه نمی‌کنم</div>
 <ul class="c-comments__user-shopping">
 <li>
 <div class="cell cell-title">رنگ:</div>
 <div class="cell color-cell">
 <span class="shopping-color-value" style="background-color: #f4fcff; border: 1px solid rgba(0, 0, 0, 0.25)"></span>مهتابی
                                     </div>
 </li>
 <li>
 <div class="cell cell-title">فروشنده:</div>
 <div class="cell seller-cell">
 <span class="o-text-blue">دیجی‌کالا</span>
 </div>
 </li>
 </ul>
 </div>, <div class="aside">
 <ul class="c-comments__user-shopping">
 <li>
 <div class="ce

In [38]:
import pandas as pd

In [68]:
df = pd.DataFrame(columns=["product_id", "seller", "date", "opinion"])

In [69]:
for comment in soup.find(class_="c-comments__list").findAll(class_="aside"):
    # is_buyer
    try:
        if comment.find(class_="c-comments__buyer-badge").text == "خریدار":
            is_buyer = True
    except AttributeError as e:
        is_buyer = False
        continue
        
    # seller
    try:
        seller = comment.find(class_="o-text-blue").text
    except AttributeError as e:
        seller = comment.find(class_="btn-link-spoiler").text
        
    # date
    date = comment.find(
        lambda tag: tag.name == 'div' and tag.get('class') == ['cell']
        ).text.replace("در تاریخ", "").strip()
    
    # opinion
    try:
        opinion = comment.find(class_="c-message-light").text
    except AttributeError as e:
        opinion = None
        
    df = df.append({
        'product_id': 507136,
        'seller': seller,
        'date': date,
        'opinion': opinion
    }, ignore_index=True)
        
        
    print("is_buyer: {}".format(is_buyer))
    print("seller: {}".format(seller))
    print("date: {}".format(date))
    print("opinion: {}".format(opinion))

is_buyer: True
seller: دیجی‌کالا
date: ۱۰ مرداد ۱۳۹۸
opinion: خرید این محصول را توصیه نمی‌کنم
is_buyer: True
seller: کوثرالکتریک
date: ۷ اردیبهشت ۱۳۹۸
opinion: خرید این محصول را توصیه می‌کنم
is_buyer: True
seller: پریز نت
date: ۹ اسفند ۱۳۹۷
opinion: خرید این محصول را توصیه می‌کنم
is_buyer: True
seller: دیجی‌کالا
date: ۱۰ آبان ۱۳۹۷
opinion: خرید این محصول را توصیه می‌کنم
is_buyer: True
seller: دیجی‌کالا
date: ۳۱ تیر ۱۳۹۸
opinion: خرید این محصول را توصیه می‌کنم
is_buyer: True
seller: دیجی‌کالا
date: ۸ آبان ۱۳۹۸
opinion: خرید این محصول را توصیه می‌کنم
is_buyer: True
seller: دیجی‌کالا
date: ۲۸ مرداد ۱۳۹۸
opinion: خرید این محصول را توصیه نمی‌کنم
is_buyer: True
seller: دیجی‌کالا
date: ۲۰ شهریور ۱۳۹۸
opinion: خرید این محصول را توصیه می‌کنم
is_buyer: True
seller: دیجی‌کالا
date: ۱۷ مرداد ۱۳۹۸
opinion: خرید این محصول را توصیه می‌کنم
is_buyer: True
seller: دیجی‌کالا
date: ۲۵ شهریور ۱۳۹۸
opinion: در مورد خرید این محصول مطمئن نیستم
is_buyer: True
seller: دیجی‌کالا
date: ۱۴ مرداد ۱۳۹۸
opinion: خرید

In [70]:
df.head()

,product_id,seller,date,opinion
0,507136,دیجی‌کالا,۱۰ مرداد ۱۳۹۸,خرید این محصول را توصیه نمی‌کنم
1,507136,کوثرالکتریک,۷ اردیبهشت ۱۳۹۸,خرید این محصول را توصیه می‌کنم
2,507136,پریز نت,۹ اسفند ۱۳۹۷,خرید این محصول را توصیه می‌کنم
3,507136,دیجی‌کالا,۱۰ آبان ۱۳۹۷,خرید این محصول را توصیه می‌کنم
4,507136,دیجی‌کالا,۳۱ تیر ۱۳۹۸,خرید این محصول را توصیه می‌کنم
